# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
df = pd.read_csv("./output_data/final_cities.csv")
df.head()

,Unnamed: 0,City,Cloudiness (%),Country,Date,Humidity (%),Lat,Lon,Max Temp (F),WindSpeed (mph)
0,0,hasaki,75,JP,1603149160,100,35.73,140.83,51.80,6.93
1,1,bredasdorp,100,ZA,1603149009,62,-34.53,20.04,59.00,8.05
2,2,avarua,10,CK,1603149174,78,-21.21,-159.78,78.80,6.93
3,3,jamestown,90,US,1603148855,100,42.10,-79.24,51.01,4.70
4,4,bluff,88,NZ,1603148928,84,-46.60,168.33,50.00,8.99


In [5]:
cleaned_df=df.drop(['City', 'Unnamed: 0', 'Cloudiness (%)', 'Country','Date','Humidity (%)','Max Temp (F)', 'WindSpeed (mph)'], axis=1)
cleaned_df.head()

,Lat,Lon
0,35.73,140.83
1,-34.53,20.04
2,-21.21,-159.78
3,42.10,-79.24
4,-46.60,168.33


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# Configure gmaps library with API key - creates the base map
#gmaps.configure(api_key=gkey)

In [11]:
# Create a humidity Heatmap layer

#locations accepts 2 columns of df or 2d arrays
locations=cleaned_df
#print(locations)
weights=df["Humidity (%)"]
#print(weights)

fig=gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights,dissipating=False, max_intensity=100, point_radius = 1))

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# cloudiness=0
cloud_df=df.loc[df['Cloudiness (%)'] == 0]
#print(cloud_df)

#windspeed less than 10mph
windspd_df=cloud_df.loc[cloud_df['WindSpeed (mph)'] >10]
#print(windspd_df)

# maxtemp greater than 65 lower than 72
maxtemp_df=windspd_df.loc[windspd_df['Max Temp (F)'] >65 ]
#print(maxtemp_df)

# maxtemp lower than 72
final_df=maxtemp_df.loc[maxtemp_df['Max Temp (F)'] <72 ].reset_index()
final_df=final_df.drop(['index', 'Unnamed: 0'], axis=1)
final_df

,City,Cloudiness (%),Country,Date,Humidity (%),Lat,Lon,Max Temp (F),WindSpeed (mph)
0,cidreira,0,BR,1603148965,91,-30.18,-50.21,67.77,15.59
1,inhambane,0,MZ,1603149663,84,-23.86,35.38,69.93,11.92
2,sabha,0,LY,1603149672,60,27.04,14.43,70.84,17.34
3,san rafael,0,AR,1603149601,23,-34.62,-68.33,69.87,10.42


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df=final_df.drop(['Cloudiness (%)', 'Country', 'Date', 'Humidity (%)', 'Max Temp (F)', 'WindSpeed (mph)'], axis=1)
hotel_df["Hotel Name"]=""
hotel_df

,City,Lat,Lon,Hotel Name
0,cidreira,-30.18,-50.21,
1,inhambane,-23.86,35.38,
2,sabha,27.04,14.43,
3,san rafael,-34.62,-68.33,


In [ ]:
# hotels within 5000 meters
#Nearby search request using places API-https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "rankby": "distance",
     "keyword": "lodging",
     "key": gkey
 }
# # use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    params["location"]=str(row["Lat"]) + "," + str(row["Lon"])
    print(params)
    response = requests.get(base_url, params=params).json()
    print(response)
#     # get location from df
#     location = row['City']

#     # add keyword to params dict
#     params['keyword'] = location

#     # assemble url and make API request
#     #print(f"Retrieving Results for Index {index}: {city}.")
   
    
#     # extract results
#     results = response['results']
    
#     try:
#         print(f"Closest hotel from {city} is {results[0]['hotelname']}.")
        
#         hotel_df.loc[index, 'Hotel Name'] = results[0]['hotelname']
       
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]